In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv

from terrain_nerf.utils import pc_plot_trace
from terrain_nerf.feature_map import FeatureMap, px_to_global, depth_to_global

# autoreload
%load_ext autoreload
%autoreload 2

Load global image

In [ ]:
# Global image

UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, 258070.0, 24860.0])

global_img = cv.imread('../data/airsim/images/test_scenario.png')
fig = px.imshow(global_img)
fig.update_layout(width=1600, height=800)
fig.show()

In [ ]:
# Create feature map
start_px = (271, 257)
goal_px = (170, 808)

feat_map = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)

Local rover image and depth image

In [ ]:
# Rover image taken from start location
rover_img = image.imread('../data/airsim/images/front_1688104627.4594698.png')
depth = np.load('../data/airsim/images/depth_1688104615.1395848.npy')

depth_thresh = 50  # meters
depth_img = depth.copy()
depth_img[depth > depth_thresh] = depth_thresh

# Show rover image and depth image side by side
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(rover_img)
ax[1].imshow(depth_img)
plt.show()

In [ ]:
cam_params = {'w': 800,
              'h': 600,
              'cx': 400, 
              'cy': 300, 
              'fx': 400, 
              'fy': 300}

In [ ]:
from terrain_nerf.utils import euler_to_R

R = euler_to_R(0, 0, np.radians(0))
#R = np.eye(3)

In [ ]:
cam_pose = np.hstack((R, np.array([[0], [0], [0]])))

Local to global association

In [ ]:
# Plot global image with local points overlaid
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
xmin, xmax, ymin, ymax = feat_map.bounds
ax.imshow(global_img, extent=[xmin, xmax, ymax, ymin])

# For each pixel in rover image, get feature from global image
G = depth_to_global(depth, cam_pose, cam_params, depth_thresh=100, patch_size=5)
ax.scatter(G[:,0], G[:,1], c=G[:,2], s=1)

plt.show()

Calculate local costs

In [ ]:
fig = go.Figure(data=pc_plot_trace(G[:,:3], color=G[:,2]))
fig.update_layout(width=1500, height=900, scene_aspectmode='data')
fig.update_layout(scene={'yaxis': {'autorange': 'reversed'}})
fig.show()

In [ ]:
G = depth_to_global(depth, cam_pose, cam_params, depth_thresh=40, patch_size=5)

In [ ]:
radius = 5
x, y = G[:,0], G[:,1]
max_depth = 40
costmap = np.zeros((max_depth+1, 2*max_depth+1))
for i in range(costmap.shape[0]):
    for j in range(costmap.shape[1]):
        x = max_depth - i
        y = j - max_depth
        dists = np.linalg.norm(G[:,:2] - np.array([x, y]))
        points = G[dists < radius]
        costmap[i,j] = np.mean(points[:,2])
        #costmap[i,j] = np.sum((x - i)**2 + (y - (j-max_depth))**2 < radius**2)

In [ ]:
# For each pixel, calculate variance of patch of points around it
# Grid up space in XY
# For each grid cell, calculate variance of points in that cell
# Plot variance as heatmap
max_depth = 40
costmap = np.zeros((max_depth+1, 2*max_depth+1))
bins = {}
scale = 1
start_px = (max_depth, max_depth)
for x, y, z in G[:,:3]:
    x_idx = start_px[0] - int(x / scale)
    y_idx = start_px[1] + int(y / scale)
    if (x_idx, y_idx) not in bins:
        bins[(x_idx, y_idx)] = [z]
    else:
        bins[(x_idx, y_idx)].append(z)
    costmap[x_idx, y_idx] = 1

cost_vals = []
for k, v in bins.items():
    cost = np.var(v)
    costmap[k] = cost
    cost_vals.append([max_depth - k[0], k[1] - max_depth, cost])

In [ ]:
plt.imshow(costmap)
plt.colorbar()

Feature association

In [ ]:
feat_map.cluster()
cost = feat_map.generate_costmap()

In [ ]:
feat_map.cluster()
cost = feat_map.generate_costmap()

# Show image and grayscale side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(global_img)
im = ax[1].imshow(cost, cmap='viridis')
plt.colorbar(im, ax=ax[1], fraction=0.05, aspect=10)
plt.show()

In [ ]:
cluster_mask = feat_map.labels == 0
local_mask = np.zeros_like(cost)
local_cost_mat = np.zeros_like(cost)

# for x, y, z, _, _ in G:
#     i, j = feat_map.global_to_img(x, y)
#     local_mask[i, j] = 1
#     local_cost_mat[i, j] = 1.0

for x, y, c in cost_vals:
    i, j = feat_map.global_to_img(x, y)
    local_mask[i, j] = 1
    local_cost_mat[i, j] = c

# Update cost for cluster
mask = cluster_mask * local_cost_mat
avg_cost = np.mean(mask[mask > 0])
cost[cluster_mask] = avg_cost

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
im = ax[0].imshow(cost, cmap='viridis')
ax[1].imshow(cluster_mask * local_mask)

In [ ]:
avg_cost

In [ ]:
# For pixel (i,j) in rover image, get feature from global image

for x, y, z, i, j in G:
    local_feature = rover_img[int(i), int(j)]
    global_feature = feat_map.get_img_feature(x, y)
    similarity = np.dot(local_feature, global_feature)
    #print(similarity) 